# Overview
[論文](https://interactiveaudiolab.github.io/project/ised.html) にあるweightを試してみる

In [1]:
import numpy as np
import pandas as pd
import librosa

# Const

# Import everything I need :)

# My function

In [2]:
def feature_extraction(wav):
    feats = librosa.feature.mfcc(wav, sr=sr, n_fft=2048, hop_length=1024, n_mels=12).T
    feat_names = [f'mfcc_{i+1}' for i in range(feats.shape[1])]
    return feats, feat_names

def weighted_euclidean_distance(a,b,w=None):
    '''
    重み付きユークリッド距離の計算
    '''
    if w is None:
        distance = np.linalg.norm(a-b)
    else:
        distance = np.sqrt((np.power(a-b,2)*w).sum())
    return distance




# Preparation

In [3]:
path = librosa.util.example_audio_file()
signal, sr = librosa.load(path, sr=None)

# EDA

feature extraction

In [4]:
feats, feat_names = feature_extraction(signal)
df_feat = pd.DataFrame(feats, columns=feat_names)
df_feat.head()

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12
0,-212.081039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-212.081039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-160.641159,49.521942,13.455418,3.512819,6.613252,4.336565,3.701090,6.923334,6.947918,4.608919,2.748719,1.054068
3,-102.801865,46.821899,10.864933,8.146544,3.715330,3.729124,2.970446,2.644935,2.137774,2.442241,2.053600,1.135120
4,-112.086372,62.333778,14.554937,4.287053,-0.336102,2.677585,3.859431,3.544301,2.320043,2.177028,1.910837,1.199781


In [5]:
n_row = len(df_feat)
df_feat['label'] = 'Positive'
df_feat.iloc[0:1000, :]['label'] = 'Negative'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df_feat['label'].value_counts()

Positive    1647
Negative    1000
Name: label, dtype: int64

<br>

weghit の計算

- 定義は以下

$$
w(i)=\frac{\left(\operatorname{avg}\left(f_{i}^{p}\right)-\operatorname{avg}\left(f_{i}^{n}\right)\right)^{2}}{\operatorname{std}\left(f_{i}^{p}\right)^{2}+\operatorname{std}\left(f_{i}^{n}\right)^{2}}
$$

In [7]:
idxs = df_feat['label'].values == 'Positive'
df_p = df_feat[idxs].drop(['label'], axis=1)
idxs = df_feat['label'].values == 'Negative'
df_n = df_feat[idxs].drop('label', axis=1)

In [8]:
posi_mean = df_p.mean(axis=0)
posi_std = np.std(df_p, axis=0)
nega_mean = df_n.mean(axis=0)
nega_std = np.std(df_n, axis=0)
w = (posi_mean - nega_mean)**2/ (posi_std ** 2 + nega_std ** 2)

In [9]:
distance = np.sqrt((np.power(posi_mean - nega_mean, 2)*w).sum())

In [13]:
mfcc = librosa.feature.mfcc(signal, sr=sr, n_fft=2048, hop_length=1024, n_mels=12).T
mfcc.shape

(2647, 12)

In [18]:
roll_off = librosa.feature.spectral_rolloff(y=signal, n_fft=2048, hop_length=1024).T
roll_off.shape

(2647, 1)

In [20]:
zcr = librosa.feature.zero_crossing_rate(y=signal, hop_length=1024).T
zcr.shape

(2647, 1)